In [ ]:
# загружаем набор данных со снеками

!pip install -q datasets

from datasets import load_dataset
dataset = load_dataset("Matthijs/snacks")
print(dataset)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.5 MB/s eta 0:00:00


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset snacks downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___snacks/default/0.0.1/c0ce49075aa469a098a5f2e3455941c894e02e1c9bf642d4d33e6c51460ff590. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4838
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 952
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 955
    })
})


In [ ]:
# Создадим словарь, в котором сопоставим название снека с числом (для проведения классификации)
labels = dataset["train"].features["label"].names
num_labels = len(dataset["train"].features["label"].names)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(label2id)
print(id2label)

{'apple': 0, 'banana': 1, 'cake': 2, 'candy': 3, 'carrot': 4, 'cookie': 5, 'doughnut': 6, 'grape': 7, 'hot dog': 8, 'ice cream': 9, 'juice': 10, 'muffin': 11, 'orange': 12, 'pineapple': 13, 'popcorn': 14, 'pretzel': 15, 'salad': 16, 'strawberry': 17, 'waffle': 18, 'watermelon': 19}
{0: 'apple', 1: 'banana', 2: 'cake', 3: 'candy', 4: 'carrot', 5: 'cookie', 6: 'doughnut', 7: 'grape', 8: 'hot dog', 9: 'ice cream', 10: 'juice', 11: 'muffin', 12: 'orange', 13: 'pineapple', 14: 'popcorn', 15: 'pretzel', 16: 'salad', 17: 'strawberry', 18: 'waffle', 19: 'watermelon'}


In [ ]:
import torch
import torch.nn as nn

# Создадим изображение для прогонки через модель, чтобы провереть что все размерности
toy_img = torch.rand(1, 3, 48, 48)

# Определим параметры для сверточной сети
num_channels = 3
hidden_size = 768
patch_size = 16

# Conv 2D - слой свертки
projection = nn.Conv2d(num_channels, hidden_size, kernel_size=patch_size,
             stride=patch_size)

# Подаем картинку в модель, чтобы посмотреть, что она определена корректно
out_projection = projection(toy_img)

print(f'Original image size: {toy_img.size()}')
print(f'Size after projection: {out_projection.size()}')

Original image size: torch.Size([1, 3, 48, 48])
Size after projection: torch.Size([1, 768, 3, 3])


In [ ]:
# Распремляем изображение после прогонки через сверточный слой

patch_embeddings = out_projection.flatten(2).transpose(1, 2)
print(f'Patch embedding size: {patch_embeddings.size()}')

Patch embedding size: torch.Size([1, 9, 768])


In [ ]:
# Определим токен [CLS], он необходим для корректной подачи данных в модель трансформера
batch_size = 1
cls_token = nn.Parameter(torch.randn(1, 1, hidden_size))
cls_tokens = cls_token.expand(batch_size, -1, -1)

# Добавим токен к вектору изображения
patch_embeddings = torch.cat((cls_tokens, patch_embeddings), dim=1)
print(f'Patch embedding size: {patch_embeddings.size()}')

Patch embedding size: torch.Size([1, 10, 768])


In [ ]:
# Определим эмбеддинги с информацией о позиции патча (кусочка картинки) в исходном изображении
position_embeddings = nn.Parameter(torch.randn(batch_size, 10, hidden_size))

# Добавим эмбеддинг к вектору изображения
input_embeddings = patch_embeddings + position_embeddings
print(f'Input embedding size: {input_embeddings.size()}')

Input embedding size: torch.Size([1, 10, 768])


In [ ]:
# Определим параметры для модели ViT
num_heads = 12
num_layers = 12

# Реализуем один блок энкодера
transformer_encoder_layer = nn.TransformerEncoderLayer(
           d_model=hidden_size, nhead=num_heads,
           dim_feedforward=int(hidden_size * 4),
           dropout=0.1)

# Объединими несколько блоков для построения модели
transformer_encoder = nn.TransformerEncoder(
           encoder_layer=transformer_encoder_layer,
           num_layers=num_layers)

# Подадим тестову картинку в модель для проверки корректности
output_embeddings = transformer_encoder(input_embeddings)
print(f' Output embedding size: {output_embeddings.size()}')

# На данном этапе мы самостоятельно построили модель ViT, однако так как модель достаточно
# большая для ее обучения потребуется значительное количество вычислительных ресурсов и данных, поэтому
# для классификации рассматриваемого датасета со снеками возьмем предобученную подель, которую сможем дообучить на наш датасет

 Output embedding size: torch.Size([1, 10, 768])


In [ ]:
!pip install transformers

from transformers import ViTModel

# Загрузим модель по ссылке
model_checkpoint = 'google/vit-base-patch16-224-in21k'
model = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)

# Пример входного изображения для проверки корректности скачивания
input_img = torch.rand(batch_size, num_channels, 224, 224)

# Прогоним картинку через модель
output_embedding = model(input_img)
print(output_embedding)
print(f"Ouput embedding size: {output_embedding['last_hidden_state'].size()}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.0070, -0.2675,  0.0940,  ...,  0.1139,  0.0424, -0.3527],
         [ 0.0944, -0.0793,  0.0593,  ..., -0.0565,  0.1158, -0.2990],
         [-0.0566, -0.3674,  0.0316,  ...,  0.0399,  0.2476, -0.3033],
         ...,
         [-0.1107, -0.2002, -0.0711,  ..., -0.0773,  0.0682, -0.3492],
         [ 0.0461, -0.3045,  0.1333,  ..., -0.0110,  0.0362, -0.2747],
         [ 0.1926, -0.1676,  0.0333,  ..., -0.0989,  0.0818, -0.2909]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=None, hidden_states=None, attentions=None)
Ouput embedding size: torch.Size([1, 197, 768])


In [ ]:
num_labels = 20

# Определим классификатор, который будет состоять из одного линейного слоя (всего классифицируем на 20 классов)
classifier = nn.Linear(hidden_size, num_labels)

# Проверим, что задали слой верно
output_classification = classifier(output_embedding['last_hidden_state'][:, 0, :])
print(f"Output embedding size: {output_classification.size()}")


Output embedding size: torch.Size([1, 20])


In [ ]:
# Импортируем библиотеки для работы с массивами, тензорами, моделью ViT и изображениями

import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Создадим класс Dataset, с помощью которого проведем предобработку изображений (обрежем и нормализуем)
# А также для формирования данных типа Dataset для более удобной подачи в модель

class ImageDataset(torch.utils.data.Dataset):

  def __init__(self, input_data):

      self.input_data = input_data
      # Transform input data
      self.transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
        ])

  def __len__(self):
      return len(self.input_data)

  def get_images(self, idx):
      return self.transform(self.input_data[idx]['image'])

  def get_labels(self, idx):
      return self.input_data[idx]['label']

  def __getitem__(self, idx):
      # Get input data in a batch
      train_images = self.get_images(idx)
      train_labels = self.get_labels(idx)

      return train_images, train_labels

In [ ]:
# Добавим к предобученной части модели ViT построенный нами линейный слой классификатора

class ViT(nn.Module):

  def __init__(self, config=ViTConfig(), num_labels=20,
               model_checkpoint='google/vit-base-patch16-224-in21k'):

        super(ViT, self).__init__()

        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = (
            nn.Linear(config.hidden_size, num_labels)
        )

  def forward(self, x):

    x = self.vit(x)['last_hidden_state']
    output = self.classifier(x[:, 0, :])

    return output

In [ ]:
# Функция обучения для модели

def model_train(dataset, epochs, learning_rate, bs):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # Загружаем модель на device (необходимо при испозьзовании GPU), определяем функцию потерь и оптимизатор
    model = ViT().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)

    # Загружаем батч с изображениями
    train_dataset = ImageDataset(dataset)
    train_dataloader = DataLoader(train_dataset, num_workers=1, batch_size=bs, shuffle=True)

    # Цикл дообучения (дообучение, так как мы взяли предобученную модель)
    for i in range(epochs):
        total_acc_train = 0
        total_loss_train = 0.0

        for train_image, train_label in tqdm(train_dataloader):
            output = model(train_image.to(device))
            loss = criterion(output, train_label.to(device))
            acc = (output.argmax(dim=1) == train_label.to(device)).sum().item()
            total_acc_train += acc
            total_loss_train += loss.item()

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f'Epochs: {i + 1} | Loss: {total_loss_train / len(train_dataset): .3f} | Accuracy: {total_acc_train / len(train_dataset): .3f}')

    return model

# Гиппер параметры
EPOCHS = 10
LEARNING_RATE = 1e-4
BATCH_SIZE = 8

# Обученная модель
trained_model = model_train(dataset['train'], EPOCHS, LEARNING_RATE, BATCH_SIZE)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 605/605 [02:44<00:00,  3.67it/s]


Epochs: 1 | Loss:  0.123 | Accuracy:  0.838


100%|██████████| 605/605 [02:49<00:00,  3.56it/s]


Epochs: 2 | Loss:  0.030 | Accuracy:  0.945


100%|██████████| 605/605 [02:53<00:00,  3.50it/s]


...

Epochs: 8 | Loss:  0.005 | Accuracy:  0.991


100%|██████████| 605/605 [02:57<00:00,  3.41it/s]


Epochs: 9 | Loss:  0.009 | Accuracy:  0.980


100%|██████████| 605/605 [02:58<00:00,  3.40it/s]

Epochs: 10 | Loss:  0.011 | Accuracy:  0.978


In [ ]:
# Функция позволяющая получить класс картинки, который предсказала обученная сеть

def predict(img):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
        ])

    img = transform(img)
    output = trained_model(img.unsqueeze(0).to(device))
    prediction = output.argmax(dim=1).item()

    return prediction

In [ ]:
# Проверим точность классификации на тестовой выборке

sum = 0
for i in range(952):
  if predict(dataset['test'][i]['image']) == dataset['test'][i]['label']:
    sum += 1

print(f'Тестовая точность:{sum/952}')

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Тестовая точность:0.8203781512605042


In [ ]:
print(5)